In [1]:
import pickle
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

pickle_in = open("C:/Users/Jitender kumar/PycharmProjects/Face_recognition/training_img.pickle","rb")
trained_img = pickle.load(pickle_in)

pickle_in = open("C:/Users/Jitender kumar/PycharmProjects/Face_recognition/training_label.pickle","rb")
trained_label = pickle.load(pickle_in)

Using TensorFlow backend.


In [2]:
pickle_in = open("C:/Users/Jitender kumar/PycharmProjects/Face_recognition/test_img.pickle","rb")
test_img = pickle.load(pickle_in)

pickle_in = open("C:/Users/Jitender kumar/PycharmProjects/Face_recognition/test_label.pickle","rb")
test_label = pickle.load(pickle_in)

In [3]:
import numpy as np

IMG_SIZ = (182,182)
normalize = np.ones(IMG_SIZ)*255
trained_img = trained_img/normalize
test_img = test_img/normalize

print(np.shape(trained_img))
print(np.shape(test_img))

(10000, 182, 182)
(3280, 182, 182)


In [6]:
# finish possible remaining session
#sess.close()

#Start interactive session
sess = tf.InteractiveSession()

In [7]:
width = 182 # width of the image in pixels 
height = 182 # height of the image in pixels
flat = width * height # number of pixels in one image 
class_output = 1 # number of possible classifications for the problem

In [8]:
x  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

In [9]:
x_image = tf.reshape(x, [-1,182,182,1])  
x_image

<tf.Tensor 'Reshape:0' shape=(?, 182, 182, 1) dtype=float32>

In [10]:
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs
print(W_conv1[0])

Tensor("strided_slice:0", shape=(5, 1, 32), dtype=float32)


In [11]:
convolve1= tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1

In [12]:
h_conv1 = tf.nn.relu(convolve1)

In [13]:
conv1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
conv1

<tf.Tensor 'MaxPool:0' shape=(?, 91, 91, 32) dtype=float32>

In [14]:
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64])) #need 64 biases for 64 outputs

In [15]:
convolve2= tf.nn.conv2d(conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2

In [16]:
h_conv2 = tf.nn.relu(convolve2)

In [17]:
conv2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') #max_pool_2x2
conv2

<tf.Tensor 'MaxPool_1:0' shape=(?, 46, 46, 64) dtype=float32>

In [18]:
layer2_matrix = tf.reshape(conv2, [-1, 46 * 46 * 64])

In [19]:
W_fc1 = tf.Variable(tf.truncated_normal([46 * 46 * 64, 2], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[2])) # need 1024 biases for 1024 outputs

In [20]:
fcl = tf.matmul(layer2_matrix, W_fc1) + b_fc1

In [21]:
h_fc1 = tf.nn.relu(fcl)
h_fc1

<tf.Tensor 'Relu_2:0' shape=(?, 2) dtype=float32>

In [22]:
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)
layer_drop

<tf.Tensor 'dropout/mul:0' shape=(?, 2) dtype=float32>

In [23]:
W_fc2 = tf.Variable(tf.truncated_normal([2, 1], stddev=0.1)) #1024 neurons
b_fc2 = tf.Variable(tf.constant(0.1, shape=[1])) # 10 possibilities for digits [0,1,2,3,4,5,6,7,8,9]

In [24]:
fc=tf.matmul(layer_drop, W_fc2) + b_fc2

In [25]:
y_CNN= tf.nn.softmax(fc)
y_CNN

<tf.Tensor 'Softmax:0' shape=(?, 1) dtype=float32>

In [26]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_CNN), reduction_indices=[1]))

In [27]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [28]:
correct_prediction = tf.equal(tf.argmax(y_CNN, 1), tf.argmax(y_, 1))

In [29]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [30]:
sess.run(tf.global_variables_initializer())

In [32]:
for i in range(1100):
    batch = trained_img[i:i+30]
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, float(train_accuracy)))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

ValueError: Cannot feed value of shape (182, 182) for Tensor 'Placeholder_2:0', which has shape '(?, 33124)'